In [ ]:
# Import Libraries
import pandas as pd
import io
import numpy as np
import re

import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk.corpus import wordnet
from nltk.corpus import stopwords

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df = pd.read_csv('latest_ticket_data.csv')

In [ ]:
def get_only_chars(line):

    clean_line = ""

    line = line.replace("’", "")
    line = line.replace("'", "")
    line = line.replace("-", " ") #replace hyphens with spaces
    line = line.replace("\t", " ")
    line = line.replace("\n", " ")
    line = line.lower()

    for char in line:
        if char in 'qwertyuiopasdfghjklzxcvbnm ':
            clean_line += char
        else:
            clean_line += ' '

    clean_line = re.sub(' +',' ',clean_line) #delete extra spaces
    if clean_line[0] == ' ':
        clean_line = clean_line[1:]
    return clean_line

In [ ]:
df['Description'] = df['Description'].apply(lambda x: get_only_chars(x))

In [ ]:
df['Tokenized']=[nltk.word_tokenize(i) for i in df['Description']]

In [ ]:
df['pos_tagged'] = [nltk.pos_tag(i) for i in df['Tokenized']]

In [ ]:
df['NN_tagged'] = df['pos_tagged'].apply(lambda item:[w for w,t in item if t=='NN'])

In [ ]:
df['NN_Description'] = df.NN_tagged.map(lambda x: ' '.join(x))

In [ ]:
stop_words = stopwords.words('english')
custom_stop_words = ['hi', 'since', 'please', 'best', 'regards', 'thank', 'thanks', 'hello', 'sent', 'great', 'dear', 'help', 'kind']
time_words = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december',
              'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday', 'today' , 'yesterday', 'tomorrow',
              'hour', 'hours', 'time', 'times', 'timelines', 'date', 'day', 'days', 'am', 'pm', 'morning', 'noon', 'afternoon', 'evening',
              'night', 'winter', 'summer', 'rain', 'cold']

def remove_stop_words(text):
    pattern = re.compile(r'\b(' + r'|'.join(stop_words) + r')\b\s*')
    text = pattern.sub('', text)
    return text

def remove_custom_words(text):
    pattern = re.compile(r'\b(' + r'|'.join(custom_stop_words) + r')\b\s*')
    text = pattern.sub('', text)
    return text

def remove_time_words(text):
    pattern = re.compile(r'\b(' + r'|'.join(time_words) + r')\b\s*')
    text = pattern.sub('', text)
    return text

df['NN_Description'] = df["NN_Description"].map(lambda x: remove_stop_words(x))
df['NN_Description'] = df["NN_Description"].map(lambda x: remove_custom_words(x))
df['NN_Description'] = df["NN_Description"].map(lambda x: remove_time_words(x))

In [ ]:
LE = LabelEncoder()
df['Label'] = LE.fit_transform(df['Category'])
print(df)

                                            Description     Category  \
0     hi since recruiter lead permission approve req...  Application   
1     re expire days hi ask help update passwords co...  Application   
2     verification warning hi has got attached pleas...  Application   
3     please dear looks blacklisted receiving mails ...  Application   
4     dear modules report report cost thank much reg...  Application   
...                                                 ...          ...   
2995  sent tuesday feedback follow up conf call hell...     Database   
2996  sent monday issues hi keep getting errors whil...     Database   
2997  sent monday en working properly hi guys we hav...     Database   
2998  sent wednesday july hi please log incident for...     Database   
2999  sent tuesday july connection issues hello have...     Database   

                                              Tokenized  \
0     [hi, since, recruiter, lead, permission, appro...   
1     [re, expire

In [ ]:
df_0 = df.loc[(df['Label'] == 0)]
df_1 = df.loc[(df['Label'] == 1)]
df_2 = df.loc[(df['Label'] == 2)]
df_3 = df.loc[(df['Label'] == 3)]
df_4 = df.loc[(df['Label'] == 4)]

In [ ]:
label_0 = ['report', 'engineer', 'information', 'confluence', 'analyst']
label_1 = ['item', 'log', 'purchase', 'order', 'po']
label_2 = ['event', 'disk', 'name', 'host', 'server']
label_3 = ['card', 'floor', 'badge' , 'access', 'manager']
label_4 = ['area', 'officer', 'visitor', 'floor', 'form']
topics = [label_0,label_1,label_2,label_3,label_4]

In [ ]:
def WordnetShortestPath_labelscore(a):
  lowest_netavg=100
  lowest_label=0
  label=-1
  #print(a)
  words = nltk.word_tokenize(a)
  for z in topics:
    total=0
    counter=0
    #print(z)
    for x in z:
      count=0
      avg=0
      sum=0
      for y in words:
        if(wordnet.synsets(x) and wordnet.synsets(y)):
          syn1 = wordnet.synsets(x)[0]
          syn2 = wordnet.synsets(y)[0]
          if(syn1.pos() == 'n' and syn2.pos() == 'n'):
            #print("Shortest path between ",x," and ",y," is: ", syn1.shortest_path_distance(syn2))
            sum=sum+syn1.shortest_path_distance(syn2)
            count=count+1
      if(count==0):
        avg=0
      else:
        avg=sum/count
      total=total+avg
      counter=counter+1
      #print(sum)
      #print(count)
      #print(avg)
    if(counter==0):
      netavg=0
    else:
      netavg=total/counter
    #print(counter)
    #print("Total score for ",z," is: ",total)
    #print("Net average for ",z," is: ",netavg)
    label=label+1
    if(netavg<lowest_netavg):
      lowest_netavg=netavg
      lowest_label=label
      #print("label: ",label," has shortest path value: ",lowest_netavg)
  return lowest_label,lowest_netavg

In [ ]:
df[ ["WSP_Label","WSP_Value"] ] = df["Description"].apply(WordnetShortestPath_labelscore).apply(pd.Series)

In [ ]:
df[ ["NN_WSP_Label","NN_WSP_Value"] ] = df["NN_Description"].apply(WordnetShortestPath_labelscore).apply(pd.Series)

In [ ]:
df["WSP_Label"] = df["WSP_Label"].astype(int)

In [ ]:
df["NN_WSP_Label"] = df["NN_WSP_Label"].astype(int)

In [ ]:
df

,Description,Category,Tokenized,pos_tagged,NN_tagged,NN_Description,Label,WSP_Label,WSP_Value,NN_WSP_Label,NN_WSP_Value
0,hi since recruiter lead permission approve req...,Application,"[hi, since, recruiter, lead, permission, appro...","[(hi, NN), (since, IN), (recruiter, NN), (lead...","[hi, recruiter, lead, permission]",recruiter lead permission,0,0,9.888889,0,9.133333
1,re expire days hi ask help update passwords co...,Application,"[re, expire, days, hi, ask, help, update, pass...","[(re, NN), (expire, NN), (days, NNS), (hi, VBP...","[re, expire, help, expire, hi, expiration, ple...",expire expire expiration prolongation expire o...,0,2,9.358025,2,9.304348
2,verification warning hi has got attached pleas...,Application,"[verification, warning, hi, has, got, attached...","[(verification, NN), (warning, VBG), (hi, NN),...","[verification, hi, please, analyst, verification]",verification analyst verification,0,2,10.200000,2,9.466667
3,please dear looks blacklisted receiving mails ...,Application,"[please, dear, looks, blacklisted, receiving, ...","[(please, VB), (dear, JJ), (looks, NNS), (blac...","[kind, engineer]",engineer,0,2,9.111111,0,5.800000
4,dear modules report report cost thank much reg...,Application,"[dear, modules, report, report, cost, thank, m...","[(dear, JJ), (modules, NNS), (report, VB), (re...","[report, cost]",report cost,0,0,8.942857,0,9.000000
...,...,...,...,...,...,...,...,...,...,...,...
2995,sent tuesday feedback follow up conf call hell...,Database,"[sent, tuesday, feedback, follow, up, conf, ca...","[(sent, NN), (tuesday, NN), (feedback, NN), (f...","[sent, tuesday, feedback, conf, call, hello, s...",feedback conf call setup meeting side meeting ...,1,2,10.165517,0,9.881481
2996,sent monday issues hi keep getting errors whil...,Database,"[sent, monday, issues, hi, keep, getting, erro...","[(sent, NN), (monday, NN), (issues, NNS), (hi,...","[sent, monday, please, help]",,1,1,9.476923,0,0.000000
2997,sent monday en working properly hi guys we hav...,Database,"[sent, monday, en, working, properly, hi, guys...","[(sent, NN), (monday, NN), (en, IN), (working,...","[sent, monday, number, meeting, room, dialer, ...",number meeting room dialer pad multiple access...,1,2,9.806780,2,9.400000
2998,sent wednesday july hi please log incident for...,Database,"[sent, wednesday, july, hi, please, log, incid...","[(sent, NN), (wednesday, NN), (july, NN), (hi,...","[sent, wednesday, july, please, log, incident,...",log incident assistance mo el ticket con colom...,1,2,9.965079,2,9.413793


In [ ]:
df_0 = df.loc[(df['Label'] == 0)]
df_1 = df.loc[(df['Label'] == 1)]
df_2 = df.loc[(df['Label'] == 2)]
df_3 = df.loc[(df['Label'] == 3)]
df_4 = df.loc[(df['Label'] == 4)]

In [ ]:
accuracy_score(df['Label'], df['WSP_Label'])

0.232

In [ ]:
accuracy_score(df['Label'], df['NN_WSP_Label'])

0.2906666666666667

In [ ]:
accuracy_score(df_0['Label'], df_0['WSP_Label'])

0.39

In [ ]:
accuracy_score(df_0['Label'], df_0['NN_WSP_Label'])

0.4716666666666667

In [ ]:
accuracy_score(df_1['Label'], df_1['WSP_Label'])

0.028333333333333332

In [ ]:
accuracy_score(df_1['Label'], df_1['NN_WSP_Label'])

0.14166666666666666

In [ ]:
accuracy_score(df_2['Label'], df_2['WSP_Label'])

0.7033333333333334

In [ ]:
accuracy_score(df_2['Label'], df_2['NN_WSP_Label'])

0.55

In [ ]:
accuracy_score(df_3['Label'], df_3['WSP_Label'])

0.0016666666666666668

In [ ]:
accuracy_score(df_3['Label'], df_3['NN_WSP_Label'])

0.023333333333333334

In [ ]:
accuracy_score(df_4['Label'], df_4['WSP_Label'])

0.03666666666666667

In [ ]:
accuracy_score(df_4['Label'], df_4['NN_WSP_Label'])

0.26666666666666666

In [ ]:
def WordnetWUP_labelscore(a):
  highest_netavg=0
  highest_label=0
  label=-1
  #print(a)
  words = nltk.word_tokenize(a)
  for z in topics:
    total=0
    counter=0
    #print(z)
    for x in z:
      count=0
      avg=0
      sum=0
      for y in words:
        if(wordnet.synsets(x) and wordnet.synsets(y)):
          syn1 = wordnet.synsets(x)[0]
          syn2 = wordnet.synsets(y)[0]
          if(syn1.pos() == 'n' and syn2.pos() == 'n'):
            #print("Shortest path between ",x," and ",y," is: ", syn1.wup_similarity(syn2))
            sum=sum+syn1.wup_similarity(syn2)
            count=count+1
      if(count==0):
        avg=0
      else:
        avg=sum/count
      total=total+avg
      counter=counter+1
      #print(sum)
      #print(count)
      #print(avg)
    if(counter==0):
      netavg=0
    else:
      netavg=total/counter
    #print(counter)
    #print("Total score for ",z," is: ",total)
    #print("Net average for ",z," is: ",netavg)
    label=label+1
    if(netavg>highest_netavg):
      highest_netavg=netavg
      highest_label=label
      #print("label: ",label," has shortest path value: ",lowest_netavg)
  return highest_label,highest_netavg

In [ ]:
df[ ["WUP_Label","WUP_Value"] ] = df["Description"].apply(WordnetWUP_labelscore).apply(pd.Series)

In [ ]:
df[ ["NN_WUP_Label","NN_WUP_Value"] ] = df["NN_Description"].apply(WordnetWUP_labelscore).apply(pd.Series)

In [ ]:
df["WUP_Label"] = df["WUP_Label"].astype(int)

In [ ]:
df["NN_WUP_Label"] = df["NN_WUP_Label"].astype(int)

In [ ]:
df

,Description,Category,Tokenized,pos_tagged,NN_tagged,NN_Description,Label,WSP_Label,WSP_Value,NN_WSP_Label,NN_WSP_Value,WUP_Label,WUP_Value,NN_WUP_Label,NN_WUP_Value
0,hi since recruiter lead permission approve req...,Application,"[hi, since, recruiter, lead, permission, appro...","[(hi, NN), (since, IN), (recruiter, NN), (lead...","[hi, recruiter, lead, permission]",recruiter lead permission,0,0,9.888889,0,9.133333,0,0.290693,0,0.309684
1,re expire days hi ask help update passwords co...,Application,"[re, expire, days, hi, ask, help, update, pass...","[(re, NN), (expire, NN), (days, NNS), (hi, VBP...","[re, expire, help, expire, hi, expiration, ple...",expire expire expiration prolongation expire o...,0,2,9.358025,2,9.304348,1,0.307018,1,0.318041
2,verification warning hi has got attached pleas...,Application,"[verification, warning, hi, has, got, attached...","[(verification, NN), (warning, VBG), (hi, NN),...","[verification, hi, please, analyst, verification]",verification analyst verification,0,2,10.200000,2,9.466667,1,0.321312,2,0.298571
3,please dear looks blacklisted receiving mails ...,Application,"[please, dear, looks, blacklisted, receiving, ...","[(please, VB), (dear, JJ), (looks, NNS), (blac...","[kind, engineer]",engineer,0,2,9.111111,0,5.800000,0,0.301727,4,0.493133
4,dear modules report report cost thank much reg...,Application,"[dear, modules, report, report, cost, thank, m...","[(dear, JJ), (modules, NNS), (report, VB), (re...","[report, cost]",report cost,0,0,8.942857,0,9.000000,0,0.299117,1,0.319167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,sent tuesday feedback follow up conf call hell...,Database,"[sent, tuesday, feedback, follow, up, conf, ca...","[(sent, NN), (tuesday, NN), (feedback, NN), (f...","[sent, tuesday, feedback, conf, call, hello, s...",feedback conf call setup meeting side meeting ...,1,2,10.165517,0,9.881481,1,0.280413,4,0.301832
2996,sent monday issues hi keep getting errors whil...,Database,"[sent, monday, issues, hi, keep, getting, erro...","[(sent, NN), (monday, NN), (issues, NNS), (hi,...","[sent, monday, please, help]",,1,1,9.476923,0,0.000000,1,0.364026,0,0.000000
2997,sent monday en working properly hi guys we hav...,Database,"[sent, monday, en, working, properly, hi, guys...","[(sent, NN), (monday, NN), (en, IN), (working,...","[sent, monday, number, meeting, room, dialer, ...",number meeting room dialer pad multiple access...,1,2,9.806780,2,9.400000,2,0.279986,1,0.282970
2998,sent wednesday july hi please log incident for...,Database,"[sent, wednesday, july, hi, please, log, incid...","[(sent, NN), (wednesday, NN), (july, NN), (hi,...","[sent, wednesday, july, please, log, incident,...",log incident assistance mo el ticket con colom...,1,2,9.965079,2,9.413793,1,0.301023,1,0.304301


In [ ]:
df_0 = df.loc[(df['Label'] == 0)]
df_1 = df.loc[(df['Label'] == 1)]
df_2 = df.loc[(df['Label'] == 2)]
df_3 = df.loc[(df['Label'] == 3)]
df_4 = df.loc[(df['Label'] == 4)]

In [ ]:
accuracy_score(df['Label'], df['WUP_Label'])

0.257

In [ ]:
accuracy_score(df['Label'], df['NN_WUP_Label'])

0.306

In [ ]:
accuracy_score(df_0['Label'], df_0['WUP_Label'])

0.2866666666666667

In [ ]:
accuracy_score(df_0['Label'], df_0['NN_WUP_Label'])

0.295

In [ ]:
accuracy_score(df_1['Label'], df_1['WUP_Label'])

0.495

In [ ]:
accuracy_score(df_1['Label'], df_1['NN_WUP_Label'])

0.39

In [ ]:
accuracy_score(df_2['Label'], df_2['WUP_Label'])

0.23666666666666666

In [ ]:
accuracy_score(df_2['Label'], df_2['NN_WUP_Label'])

0.195

In [ ]:
accuracy_score(df_3['Label'], df_3['WUP_Label'])

0.045

In [ ]:
accuracy_score(df_3['Label'], df_3['NN_WUP_Label'])

0.24

In [ ]:
accuracy_score(df_4['Label'], df_4['WUP_Label'])

0.22166666666666668

In [ ]:
accuracy_score(df_4['Label'], df_4['NN_WUP_Label'])

0.41